<a href="https://colab.research.google.com/github/anilsolanki2645/Snowflake_College_Database_Automation__Solution/blob/main/Email_Alert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the required libraries
!pip install snowflake-connector-python boto3

import os
import boto3
import snowflake.connector
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 829.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.2 MB/s eta 0:00:00


In [ ]:
# AWS credentials (replace with your credentials or configure AWS CLI on your local machine)
AWS_ACCESS_KEY = '*****************'  # Avoid hardcoding credentials in production
AWS_SECRET_KEY = '************************'
REGION_NAME = 'us-east-1'  # e.g., 'us-east-1'

# Snowflake connection details (replace with your actual credentials)
SNOWFLAKE_USER = 'Anu'
SNOWFLAKE_PASSWORD = 'your_password'
SNOWFLAKE_ACCOUNT = 'bl14512.ap-southeast-1'
SNOWFLAKE_WAREHOUSE = 'COMPUTE_WH'
SNOWFLAKE_DATABASE = 'ECOM'
SNOWFLAKE_SCHEMA = 'PUBLIC'
SNS_TOPIC_ARN = 'Your_SNS_TOPIC_ARN'

# Initialize the SNS client
sns_client = boto3.client(
    'sns',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=REGION_NAME
)

In [ ]:
def check_task_status():
    # Connect to Snowflake
    conn = snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA,
        authenticator='snowflake',
        client_session_keep_alive=True,  # Keeps the session alive, helpful for longer operations
        retry_attempts=3
    )

    try:
        # Query to check the status from TASK_STATUS table
        query = """
        SELECT TASK_NAME, SCHEMA_NAME, DATABASE_NAME, STATUS, ERROR_MESSAGE, QUERY_TEXT
        FROM ECOM.PUBLIC.TASK_STATUS;
        """

        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()

        # Check the fetched results and send alerts based on specific conditions
        # Example condition: Check if the 'STATUS' column has 'FAILED' or any other failure indicator
        for row in results:
            # Modify based on your table's structure and the condition you want to monitor
            TASK_NAME, SCHEMA_NAME, DATABASE_NAME, STATUS, ERROR_MESSAGE, QUERY_TEXT = row  # Example row structure

            if STATUS == 'FAILED':  # Adjust condition based on your table structure
                message = (
                    f"Task Failure Alert :                   [1] "
                    f"TASK_NAME : {TASK_NAME}                [2] "
                    f"SCHEMA_NAME : {SCHEMA_NAME}            [3] "
                    f"DATABASE_NAME : {DATABASE_NAME}        [4] "
                    f"STATUS : {STATUS}                      [5] "
                    f"ERROR_MESSAGE : {ERROR_MESSAGE}        [5] "
                    f"QUERY_TEXT : {QUERY_TEXT}              "
                )
                print(f"Failures detected for task: {TASK_NAME}")
                send_alert(message)
            else:
                print(f"No failures detected for task: {TASK_NAME}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        cursor.close()
        conn.close()

In [ ]:
def send_alert(message):
    try:
        response = sns_client.publish(
            TopicArn=SNS_TOPIC_ARN,
            Message=message,
            Subject='Snowflake Task Status Alert'
        )
        print(f"Alert sent! Message ID: {response['MessageId']}")
    except Exception as e:
        print(f"Failed to send alert: {str(e)}")

In [ ]:
# Run the task status check
check_task_status()

No failures detected for task: TASK_SHIPPING
No failures detected for task: TASK_UPDATE_TASK_STATUS
No failures detected for task: TASK_PAYMENTS
Failures detected for task: TASK_PAYMENTS
Alert sent! Message ID: 40433e67-1426-5596-abe0-61168bd1919a
No failures detected for task: TASK_PRODUCTS
No failures detected for task: TASK_ORDERS
No failures detected for task: TASK_CUSTOMERS
No failures detected for task: TASK_ORDER_ITEMS
No failures detected for task: TASK_REVIEWS
